In [ ]:
import numpy as np
import numpy.random as rand
from functools import cache
import polars as pl
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

import matplotlib.pyplot as plt
import seaborn
import pandas as pd

In [ ]:
TARGET = "quality"
df = pl.read_csv("data/Wine_Quality_Data.csv", has_header=True)
df = df.drop("color")
df = df.head(1000)

In [ ]:
df.describe()

In [ ]:
from sklearn.model_selection import KFold
import polars as pl
import itertools

def fetch_data(df, use_split, rbf, include_mul, raw_vals):
    assert(use_split >= 0 and use_split <= 4)
    # Perform 5-fold cross validation with a deterministic seed
    kf = KFold(n_splits=4, shuffle=True, random_state=2023)
    splits = list(kf.split(df))
    # Indexing the dataframe with an array returns the appropriate splits
    df_train, df_test = df[splits[use_split][0]], df[splits[use_split][1]]
    for col in df_train.columns:
        if col == TARGET or col in rbf:
            continue
        df_train, df_test = scale_column(df_train, df_test, col)
    cols = list(df_train.columns)
    # binary = binary_columns(df_train)
    for (idx, col) in enumerate(rbf):
        if col == TARGET:
            continue
        dist = int(10 * abs(0.5 - raw_vals[idx]))
        dist = np.clip(dist, 0, 5)
        df_train, df_test = rbf_column(df_train, df_test, col, num_features=dist) # Use radial bias function
    for (idx, col) in enumerate(cols):
        if col == TARGET:
            continue
        if col not in rbf:
            dist = int(10 * abs(0.5 - raw_vals[idx]))
            dist = np.clip(dist, 0, 5)
            df_train, df_test = fourier_column(df_train, df_test, col, num_features=dist)
    for (col_a, col_b) in itertools.combinations(cols, 2):
        if col_a == TARGET or col_b == TARGET:
            continue
        if col_a not in include_mul and col_b not in include_mul:
            continue
        df_train, df_test = mul_cols(df_train, df_test, col_a, col_b)
    return df_train, df_test

def binary_columns(df):
    binary = []
    for col in df.columns:
        if df.get_column(col).n_unique() == 2:
            binary.append(col)
    return binary

def scale_column(df_train, df_test, col_name):
    '''Scale a column from 0 to 1'''
    max = df_train.get_column(col_name).max()
    min = df_train.get_column(col_name).min()
    df_train = df_train.with_columns((pl.col(col_name) - min) / (max - min))
    df_test = df_test.with_columns((pl.col(col_name) - min) / (max - min))
    return df_train, df_test

def rbf_column(df_train, df_test, col_name, num_features=3):
    '''Divide a col into 3 features using a radial basis function'''
    std = df_train.get_column(col_name).std()
    select = []
    if num_features == 2:
        low = df_train.get_column(col_name).quantile(0.25)
        high = df_train.get_column(col_name).quantile(0.75)
        select = [low, high]
    elif num_features == 3:
        low = df_train.get_column(col_name).quantile(0.25)
        middle = df_train.get_column(col_name).quantile(0.5)
        high = df_train.get_column(col_name).quantile(0.75)
        select = [low, middle, high]
    elif num_features == 4:
        low = df_train.get_column(col_name).quantile(0.2)
        middle = df_train.get_column(col_name).quantile(0.4)
        high = df_train.get_column(col_name).quantile(0.6)
        higher = df_train.get_column(col_name).quantile(0.8)
        select = [low, middle, high, higher]
    elif num_features == 5:
        low = df_train.get_column(col_name).quantile(0.15)
        middle = df_train.get_column(col_name).quantile(0.3)
        high = df_train.get_column(col_name).quantile(0.45)
        higher = df_train.get_column(col_name).quantile(0.6)
        highest = df_train.get_column(col_name).quantile(0.75)
        select = [low, middle, high, higher, highest]
    for i, val in enumerate(select):
        df_train = df_train.with_columns((-1.0 * (pl.col(col_name) - val)**2 / (2 * std)**2).exp().alias(f"{col_name}_{i}"))
        df_test = df_test.with_columns((-1.0 * (pl.col(col_name) - val)**2 / (2 * std)**2).exp().alias(f"{col_name}_{i}"))
    return df_train, df_test

def fourier_column(df_train, df_test, col_name, num_features=3):
    '''Divide a col in 3 features using math'''
    for i in range(1, num_features + 1):
        df_train = df_train.with_columns((pl.col(col_name) * pl.lit(i) * pl.lit(np.pi)).cos().alias(f"{col_name}_{i - 1}"))
        df_test = df_test.with_columns((pl.col(col_name) * pl.lit(i) * pl.lit(np.pi)).cos().alias(f"{col_name}_{i - 1}"))
    return df_train, df_test

def mul_cols(df_train, df_test, col_a, col_b):
    df_train = df_train.with_columns((pl.col(col_a) * pl.col(col_b)).alias(f"{col_a}+{col_b}"))
    df_test = df_test.with_columns((pl.col(col_a) * pl.col(col_b)).alias(f"{col_a}+{col_b}"))
    return df_train, df_test

def get_x_y(df):
    X, y = df.drop(TARGET), df.get_column(TARGET)
    return X.to_numpy(), y.to_numpy()

NUM_FEATURES = len(df.columns) - 1
df_train, df_test = fetch_data(df, 0, list(df.columns)[11:], [], np.ones(NUM_FEATURES * 2))
X_train, y_train = get_x_y(df_train)
X_test, y_test = get_x_y(df_test)
len(y_train)

In [ ]:
NUM_FEATURES

In [ ]:
a = np.array(list(df.columns[1:]) + list(df.columns[1:]))
len(a)

In [ ]:
def evaluate(input, model='tree', verbose=False): # model = tree, svm, logistic, knn
    a = np.array(list(df.columns[1:]) + list(df.columns[1:]))
    rbf = a[input > 0.5]
    df_train, df_test = fetch_data(df, 0, rbf[:NUM_FEATURES], rbf[NUM_FEATURES:], input)
    if verbose:
        print("Fetched Data")
    X_train, y_train = get_x_y(df_train)
    X_test, y_test = get_x_y(df_test)
    if verbose:
        print("Split data")

    if model == 'logistic': reg = LogisticRegression(solver="newton-cg", random_state=420)
    elif model == 'svm': reg = LinearSVC(random_state=420)
    elif model == 'tree': reg = DecisionTreeClassifier(random_state=420)
    elif model == 'knn': reg = KNeighborsClassifier()
    else: reg = None
    if verbose:
        print("Fitting Data")
    reg.fit(X_train, y_train)
    return reg.score(X_test, y_test)

In [ ]:
arr = np.ones(NUM_FEATURES * 2)
arr[NUM_FEATURES:] = 0.6
evaluate(arr)

In [ ]:
np.array(list(df.columns[1:]))

In [ ]:
list(df.columns)[2:]

In [ ]:
NUM_DIM = NUM_FEATURES * 2
LIMIT = 4

class Configuration:
    def __init__(self, array):
        self.data = np.clip(array, 0.0, 1.0)

    def __hash__(self):
        return hash(self.data.tobytes())
    
    def __eq__(self, other):
        return (self.data == other.data).all()

class Solutions:
    def __init__(self, num, model='fhsdkfhkd', ring_size=4, seed=4222023):
        self.rng = rand.Generator(rand.PCG64(seed))
        self.solutions = np.stack([self.new_solution() for _ in range(num)])
        self.employed = np.array([True] * (num // 2) + [False] * (num // 2))
        self.failures = np.zeros_like(self.employed, np.int32)
        self.onlooker = np.logical_not(self.employed) # Unemployed
        self.best_fitness = -1
        self.best_sol = None 
        self.ring_size = ring_size
        self.model = model

    def new_solution(self):
        return self.rng.random(size=NUM_DIM)
    
    def best_in_ring(self, start_idx):
        # sol = self.solutions[start_idx]
        size = len(self.solutions)
        best_idx = start_idx 
        for i in range(1, 1 + self.ring_size):
            idx = (start_idx - i) % size
            if self.fitness(self.solutions[idx]) > self.fitness(self.solutions[best_idx]):
                best_idx = idx
        for i in range(1, 1 + self.ring_size):
            idx = (start_idx + i) % size
            if self.fitness(self.solutions[idx]) > self.fitness(self.solutions[best_idx]):
                best_idx = idx
        return best_idx
        

    def random_sol(self, exclude=-1):
        rand_idx = self.rng.integers(0, self.solutions.shape[0])
        if rand_idx == exclude:
            return self.random_sol(exclude=exclude)
        else:
            return self.solutions[rand_idx]

    def get_employed(self):
        return self.solutions[self.employed]

    def get_unemployed(self):
        return self.solutions[np.logical_not(self.employed)]
        
    def get_onlooker(self):
        return self.solutions[self.onlooker]

    def get_scout(self):
        return self.solutions[self.scout]

    def fitness(self, x):
        return _fitness(Configuration(x), self.model)

    def most_fit(self):
        fit = np.array([self.fitness(x) for x in self.solutions])
        idx = fit.argmax()
        return fit[idx], self.solutions[idx]

    def update_best(self):
        best_fit, best_sol = self.most_fit()
        if best_fit > self.best_fitness:
            self.best_sol = best_sol
            self.best_fitness = best_fit

@cache
def _fitness(x: Configuration, model):
    return evaluate(x.data, model)

def employed_fn(sol: Solutions, initial_idx: int, enhanced=False):
    initial = sol.solutions[initial_idx]
    a = 0.1
    idx = sol.rng.integers(0, initial.size)
    phi = sol.rng.uniform(low=-a, high=a)
    out = np.copy(initial)
    if enhanced: 
        best_idx = sol.best_in_ring(initial_idx)
        sol_k = sol.solutions[best_idx]
    else:
        sol_k = sol.random_sol(exclude=initial_idx)
    out[idx] += phi * (out[idx] - sol_k[idx])
    out = np.clip(out, 0, 1)
    return out

def basic_onlooker(sol: Solutions, _initial_idx: int):
    employed = sol.get_employed()
    fitnesses = np.array([sol.fitness(x) for x in employed])
    total_fitness = np.sum(fitnesses)
    bee_idx = sol.rng.choice(np.arange(len(employed)), p=fitnesses/total_fitness)
    return employed_fn(sol, bee_idx)

def enhanced_onlooker(sol: Solutions, initial_idx: int):
    best_bee = sol.solutions[sol.best_in_ring(initial_idx)]
    random_bee_idx = sol.rng.choice(len(sol.solutions))
    random_bee = sol.solutions[random_bee_idx]
    a = 0.1 
    idx = sol.rng.integers(0, best_bee.size)
    phi = sol.rng.uniform(low=-a, high=a)
    out = np.copy(best_bee)
    out[idx] += phi * (out[idx] - random_bee[idx])
    out = np.clip(out, 0, 1)
    return out

def vanilla_abc(num_bees, epochs):
    return abc(num_bees, epochs, basic_onlooker, smart_scout=False, smart_employed=False)

def abc(num_bees, epochs, onlooker_fn, smart_scout=False, smart_employed=False, model='tree', quiet=True):
    sol = Solutions(num_bees, model)
    for _ in range(epochs):
        # Employed
        for idx in sol.employed.nonzero()[0]:
            candidate = employed_fn(sol, idx, smart_employed)
            if sol.fitness(candidate) > sol.fitness(sol.solutions[idx]):
                sol.solutions[idx] = candidate
                sol.failures[idx] = 0
            else:
                sol.failures[idx] += 1
        # Onlooker
        for idx in sol.onlooker.nonzero()[0]:
            candidate = onlooker_fn(sol, idx)
            if sol.fitness(candidate) > sol.fitness(sol.solutions[idx]):
                sol.solutions[idx] = candidate
        # Scout
        for idx in sol.employed.nonzero()[0]:
            if sol.failures[idx] >= LIMIT:
                sol.failures[idx] = 0
                sol.solutions[idx, :] = sol.new_solution()
            if smart_scout:
                new_fitness = sol.fitness(sol.solutions[idx])
                # Candidate U2
                r1 = sol.random_sol()
                r2 = sol.random_sol()
                best = sol.solutions[sol.best_in_ring(idx)]
                diff = r1 - r2
                weighted = sol.rng.random(diff.size) * diff
                candidate = np.copy(best)
                candidate += weighted
                u2_fit = sol.fitness(candidate)
                if u2_fit > new_fitness:
                    new_fitness = u2_fit 
                    sol.solutions[idx, :] = candidate
                # Candidate U3
                # Assume in range 0 to 1
                opposite = np.ones_like(best) - best
                u3_fit = sol.fitness(opposite)
                if u3_fit > new_fitness:
                    sol.solutions[idx, :] = opposite
            
        # Mark best
        sol.update_best()
        if not quiet:
            print(sol.best_fitness)
            print(sol.best_sol)
    return sol

In [ ]:
RUN = False

In [ ]:
import warnings

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    s = abc(100, 10, enhanced_onlooker, smart_scout=True, smart_employed=True, model='tree', quiet=False)

In [ ]:
import warnings
if RUN:
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        best = abc(80, 2**10, enhanced_onlooker, True, True)
        best_sol = best.best_sol
        best_acc = best.best_fitness

else:
    best_sol = np.array([ 1.41758502e-01,  5.36575656e-01,  1.52264166e-01,  5.22438538e-01,
        8.53801988e-01,  3.42520398e-02,  7.00347225e-02,  5.84034794e-01,
        2.87205723e+01,  1.13782776e+04, -1.35060538e+04, -8.34848191e+03,
       -2.19114797e+03, -3.10108235e+03,  1.77171210e+01, -3.28302531e+03,
       -1.82234877e+04,  3.37441394e-01, -2.34847110e+03,  8.42227212e-02,
        6.08844870e-01, -1.99673131e+02])
    best_acc = 0.724

In [ ]:
best.best_fitness

In [ ]:
# Best Random Forest 0.74 [0.52904654 0.81420687 0.64489839 0.7312731  0.27315985 0.97688487
#  0.57291268 0.28252793 0.22780821 0.6718864  0.16589953 0.81165986
#  0.91879719 0.36721561 0.20586594 0.09941241 0.58837168 0.23499118
#  0.21606769 0.41012035 0.447105   0.16644631]

In [ ]:
# Best DecisionTree 0.716 [ 0.49781814  0.27629634  0.7451372   0.57612247  0.8015136   0.07614703
#  -1.23652068  0.60414707  0.57149879  0.52726963  0.58626366 -0.51605658
#   0.52291123  0.46263772  2.53687284  0.36810732  0.49246265 -0.23834081
#   1.93182876  0.86507608  0.22388496  0.69642921]

In [ ]:
# Best DecisionTree 0.744
vals = np.array([0.47814596, 0.5348577, 0.75409125, 0.93711038, 0.93402507, 0.80108002,
 0.38969105, 0.52728978, 0.40916796, 0.67543753, 0.12029326, 0.52322336,
 0.17724371, 0.03142871, 0.95574856, 0.9284217,  0.94973273, 0.70183991,
 0.62261783, 0.22162607, 0.23216788, 0.16600173])
vals[:NUM_FEATURES]

In [ ]:
vals[NUM_FEATURES:]

In [ ]:
best_sol[NUM_FEATURES:]

In [ ]:
import pandas as pd

scores = []

for i in range(NUM_FEATURES):
    for _ in range(10):
        new_vals = np.copy(best_sol)
        r = rand.uniform()
        new_vals[i] = r
        if i == 8:
            print(new_vals.tolist())
        score = evaluate(new_vals)
        scores.append({ 'dim': i, 'score': score })
    print(i)

data = pd.DataFrame.from_records(scores)

In [ ]:
ouch = np.array([0.141758502, 0.536575656, 0.152264166, 0.522438538, 0.853801988, 0.0342520398, 0.0700347225, 0.584034794, 
                 0.226881221289003, 11378.2776, -13506.0538, -8348.48191, -2191.14797, -3101.08235, 17.717121, -3283.02531, 
                 -18223.4877, 0.337441394, -2348.4711, 
                 0.0842227212, 0.60884487, -199.673131])
okay = np.copy(ouch)
okay[8] = 0.7968510552842712
print(okay)
evaluate(ouch, "tree", verbose=True)

In [ ]:
data

In [ ]:
import seaborn

f, ax = plt.subplots()

seaborn.boxplot(data=data, x="dim", y="score", flierprops={"marker": "x"}, ax=ax)
ax.hlines(best_acc, ax.get_xlim()[0], ax.get_xlim()[1])

In [ ]:
import matplotlib.pyplot as plt
import seaborn
import pandas as pd

plt.style.use('ggplot')
COLORS = seaborn.color_palette()
f, ax = plt.subplots(1, 2, figsize=(12, 4))

scores = [0.74, 0.71, 0.67] 
models = ['Decision Tree', 'Logistic Regression', 'SVM']

plot_df = pd.DataFrame.from_dict({'Score': scores, 'Models': models})
ax[0].set_title('Accuracy of Different Models')
seaborn.barplot(data=plot_df, x='Models', y='Score', ax=ax[0], color=COLORS[0])
for container in ax[0].containers:
    ax[0].bar_label(container, fmt='%.2f')

speed = [0.2, 0.4, 0.1] 
models = ['Decision Tree', 'Logistic Regression', 'SVM']

plot_df = pd.DataFrame.from_dict({'Speed (s)': speed, 'Models': models})
ax[1].set_title('Speed of Different Models')
seaborn.barplot(data=plot_df, x='Models', y='Speed (s)', ax=ax[1], color=COLORS[1])
for container in ax[1].containers:
    ax[1].bar_label(container, fmt='%.2f')

In [ ]:
s = vanilla_abc(10, 0)
s.get_employed()

In [ ]:
s = vanilla_abc(20, 0)
s.get_employed()

In [ ]:
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import numpy as np

RUN = True

plt.style.use('ggplot')
COLORS = seaborn.color_palette()
f, ax = plt.subplots()

bees = [20, 30, 50, 80, 100]
if RUN:
    vanilla = np.array([vanilla_abc(b, 100).best_fitness for b in bees])
    enhanced_abc = np.array([abc(b, 100, enhanced_onlooker, True, True).best_fitness for b in bees])
else:
    enhanced_abc = np.array([0.67, 0.69, 0.702, 0.71, 0.728])
    vanilla = np.array([0.62, 0.66, 0.67, 0.702, 0.71])

method = ['Vanilla' for _ in range(5)] + ['Enhanced' for _ in range(5)]
plot_df = pd.DataFrame.from_dict({'Accuracy': np.concatenate([vanilla, enhanced_abc]), 'Method': method, '# of Bees': bees + bees})
ax.set_title('Vanilla vs Enhanced ABC')
seaborn.lineplot(data=plot_df, x='# of Bees', y='Accuracy', hue='Method', ax=ax, palette=COLORS[0:2])
seaborn.move_legend(ax, 'lower center')
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f')

In [ ]:
import matplotlib.pyplot as plt
import seaborn
import pandas as pd
import numpy as np

plt.style.use('ggplot')
COLORS = seaborn.color_palette()
f, ax = plt.subplots()
epochs = [2**(i + 3) for i in range(8)]
if RUN:
    vanilla = np.array([vanilla_abc(80, e).best_fitness for e in epochs])
    enhanced_abc = np.array([abc(80, e, enhanced_onlooker, True, True).best_fitness for e in epochs])
else:
    # Saved values from our 
    enhanced_abc = np.array([0.696, 0.696, 0.708, 0.708, 0.708, 0.708, 0.708, 0.724])
    vanilla = np.array([0.7  , 0.7  , 0.7  , 0.7  , 0.7  , 0.704, 0.704, 0.712])

method = ['Vanilla' for _ in range(len(epochs))] + ['Enhanced' for _ in range(len(epochs))]
plot_df = pd.DataFrame.from_dict({'Accuracy': np.concatenate([vanilla, enhanced_abc]), 'Method': method, 'Epochs': epochs + epochs})
ax.set_title('Vanilla vs Enhanced ABC')
ax.set_xscale("log")
seaborn.lineplot(data=plot_df, x='Epochs', y='Accuracy', hue='Method', ax=ax, palette=COLORS[0:2])
seaborn.move_legend(ax, 'upper center')
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f')

In [ ]:
vanilla

In [ ]:
enhanced_abc

In [ ]:
epochs

In [ ]:
plot_df